## 유통정보 - 주요 품목 가격
데이터 호출 URL, 파라미터 설명, XML태그 설명 등의 내용을 포함한 표

데이터 호출 URL	http://www.garak.co.kr/publicdata/dataOpen.do?id=3265&passwd=Wjdwlr!268&dataid=data4&pagesize=10
&pageidx=1&portal.templet=false&p_ymd=20140430&p_jymd=20140429&d_cd=2&p_jjymd=20130429
&p_pos_gubun=1&pum_nm=

파라미터 설명	- id :  (수정불가)
- passwd :  (수정불가)
- p_ymd : 검색일자(8자리)
- p_jymd : 전일날짜(8자리)
- d_cd : 부류구분(청과:2 , 수산: 3)
- p_jjymd : 전년도날짜(8자리)
- p_pos_gubun : 시장구분(가락시장 : 1, 양곡시장 : 2, 강서시장(경매) :3, 강서시장(시장도매인) : 9 )
- pum_nm : 품목명(예: 사과, 배 등 )

XML태그	- PUM_NM_A : 품목명
- PUM_CD : 품목코드
- G_NAME_A : 등급
- UNIT_QTY : 거래수량
- U_NAME : 거래단위수량
- AV_P_A : 평균가격
- PAV_P_A : 전일평균가격
- PAV_PY_A : 전년가격
- A_B : 전일대비등락율
- A_C : 검색일전년대비등락율
- E_NAME : 등급
- F_NAME : 거래단위
- GRADE_CD : 등급단위
- UNIT_CD : 중량단위

In [221]:
import requests
import pandas as pd
import numpy as np
#https://www.garak.co.kr/price/OZViewer.do
url = 'http://www.garak.co.kr/publicdata/dataOpen.do'
params = {
('id','3265'),
('passwd','Wjdwlr!268'),
('p_ymd','20220915'), #검색일자
('p_jymd','20220914'), #전일날짜
('d_cd','2'), #부류구분(청과:2,수산:3)
('p_jjymd','20210915'), #전년도날짜(8자리)
('p_pos_gubun','1'), #시장구분(가락시장 : 1, 양곡시장 : 2, 강서시장(경매) :3, 강서시장(시장도매인) : 9 )
('pum_nm','고구마'), #품목명(예: 사과, 배 등 )
#밑에 전달값은 URL 내용을 가져온 것
('portal.templet','false',),
('pagesize', '10'), #페이지 사이즈, max 10개
('pageidx', '1'),   #페이지 번호, 번호 늘릴 수록 더 받아옴
('dataid','data4')  #데이터구분값
}
response = requests.get(url,params)
print(response)

<Response [200]>


In [222]:
content = response.text
content = content.split('\n')
df = []
row = []
for i,j in enumerate(content):
    position_st = j.find('CDATA[')
    position_en = j.find(']]')
    # CDATA가 없는 경우는 반복문 넘어가기
    if position_st == -1:
        continue
    # 잡히는 경우 해당 데이터 수집
    else:
        txt = j[position_st+6:position_en].strip()
        row.append(txt)
    #마지막 컬럼에 도달했다면 row를 초기화
    if j.startswith('<UNIT_CD>'):
        df.append(row)
        row = []
df_raw = pd.DataFrame(data = np.array(df), columns = ['품목명','품목코드','등급','거래수량','거래단위수량','평균가격','전일평균가격','전년가격','전일대비등락율','검색일전년대비등락율','등급','거래단위','등급단위','중량단위'])

df_raw['전일대비등락율'] = df_raw['전일대비등락율'].astype('float').round(decimals = 1)
df_raw['검색일전년대비등락율'] = df_raw['검색일전년대비등락율'].astype('float').round(decimals = 1)

df_raw

,품목명,품목코드,등급,거래수량,거래단위수량,평균가격,전일평균가격,전년가격,전일대비등락율,검색일전년대비등락율,등급,거래단위,등급단위,중량단위
0,고구마,15100,특,10,10키로상자,34305,33318,40680,103.0,84.3,특,키로상자,0,01
1,고구마,15100,상,10,10키로상자,24608,23642,26971,104.1,91.2,상,키로상자,1,01
2,고구마,15100,중,10,10키로상자,17257,16401,18006,105.2,95.8,중,키로상자,2,01
3,고구마,15100,하,10,10키로상자,9324,8961,7279,104.1,128.1,하,키로상자,3,01
4,고구마(호박밤),15103,특,10,10키로상자,31078,30060,33983,103.4,91.5,특,키로상자,0,01
5,고구마(호박밤),15103,상,10,10키로상자,24585,22403,25693,109.7,95.7,상,키로상자,1,01
6,고구마(호박밤),15103,중,10,10키로상자,17910,16308,17979,109.8,99.6,중,키로상자,2,01
7,고구마(호박밤),15103,하,10,10키로상자,11046,9577,7204,115.3,153.3,하,키로상자,3,01
8,호박 고구마,15111,특,10,10키로상자,34947,33592,41434,104.0,84.3,특,키로상자,0,01
9,호박 고구마,15111,상,10,10키로상자,25396,23862,27597,106.4,92.0,상,키로상자,1,01


## 해볼수 있는 것
1. 연중 청과,수산물에 대한 연 평균가격 도시
2. 현재 제철(가격)인 품목의 현재가 도시 & 이메일 알림
